In [2]:
import pygame
import chess
import random

# Tahta boyutları
SQUARE_SIZE = 64
BOARD_SIZE = SQUARE_SIZE * 8  # 8 satır ve 8 sütun

# Pygame'i başlat
pygame.init()
screen = pygame.display.set_mode((BOARD_SIZE, BOARD_SIZE))
pygame.display.set_caption("Xiast Chess")

# Renk tanımlamaları
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GRAY = (200, 200, 200)
DARK_GRAY = (150, 150, 150)
GREEN = (0, 128, 0)
BLUE = (0, 0, 255)
ORANGE = (255, 165, 0)

# Menü seçenekleri
menu_options = ["Yeni Oyun", "Çıkış"]
menu_colors = [BLACK] * len(menu_options)

# Resim yükleme ve boyutlandırma fonksiyonu
def load_and_scale_image(file_path):
    image = pygame.image.load(file_path)
    return pygame.transform.scale(image, (SQUARE_SIZE, SQUARE_SIZE))  # Resmi boyutlandır

# Taş resim isimlerini döndür
def get_piece_image_name(piece):
    piece_dict = {
        chess.PAWN: 'P',
        chess.ROOK: 'R',
        chess.KNIGHT: 'N',
        chess.BISHOP: 'B',
        chess.QUEEN: 'Q',
        chess.KING: 'K'
    }
    return f"{'w' if piece.color == chess.WHITE else 'b'}{piece_dict[piece.piece_type]}.png"

# Tahtayı çizme fonksiyonu
def draw_board(board):
    for i in range(8):  # 8 satır
        for j in range(8):  # 8 sütun
            square_color = WHITE if (i + j) % 2 == 0 else (125, 135, 150)
            pygame.draw.rect(screen, square_color, (j * SQUARE_SIZE, i * SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE))  # Kareleri çiz

            piece_position = board.piece_at(chess.square(j, 7 - i))  # Taşların konumunu ayarla
            if piece_position is not None:
                piece_image = load_and_scale_image(f"pieces/{get_piece_image_name(piece_position)}")
                screen.blit(piece_image, (j * SQUARE_SIZE, i * SQUARE_SIZE))  # Taşları kareye yerleştir

# Taş hareket ettirme fonksiyonu
def move_piece(board, start_square, end_square):
    move = chess.Move(start_square, end_square)
    if move in board.legal_moves:  # Geçerli bir hamle olup olmadığını kontrol et
        board.push(move)  # Hamleyi yap
        return True  # Hamle başarıyla yapıldı
    else:
        print("Geçersiz hamle!")
        return False  # Hamle yapılamadı

# Kazanan rengi gösterme fonksiyonu
def show_winner(winner_color):
    font = pygame.font.Font(None, 74)
    text = font.render(f"{'Beyaz' if winner_color == chess.WHITE else 'Siyah'} kazandı!", True, GREEN)
    screen.blit(text, (BOARD_SIZE // 2 - text.get_width() // 2, BOARD_SIZE // 2 - text.get_height() // 2))

# Menü ekranını çizme fonksiyonu
def draw_menu(selected_option):
    screen.fill(WHITE)  # Ekranı temizle
    font = pygame.font.Font(None, 74)
    title_text = font.render("Satranç Oyunu", True, BLACK)
    screen.blit(title_text, (BOARD_SIZE // 2 - title_text.get_width() // 2, BOARD_SIZE // 4))

    for i, option in enumerate(menu_options):
        option_text = font.render(option, True, WHITE)
        option_rect = option_text.get_rect(center=(BOARD_SIZE // 2, BOARD_SIZE // 2 + i * 100))

        # Seçenek üzerine gelindiğinde arka plan rengini değiştir
        background_color = BLUE if selected_option == i else ORANGE
        pygame.draw.rect(screen, background_color, option_rect.inflate(20, 20), border_radius=10)
        screen.blit(option_text, option_rect)  # Seçeneği çiz

# Menü seçim fonksiyonu
def menu_selection(pos):
    for i, option in enumerate(menu_options):
        option_rect = pygame.Rect(0, 0, 0, 0)  # Boş bir dikdörtgen tanımla
        option_text = pygame.font.Font(None, 74).render(option, True, BLACK)
        option_rect = option_text.get_rect(center=(BOARD_SIZE // 2, BOARD_SIZE // 2 + i * 100))

        if option_rect.collidepoint(pos):  # Fare tıklama noktasının menü seçeneğiyle çakışıp çakışmadığını kontrol et
            return i
    return -1  # Hiçbir seçeneğe tıklanmadı

# Ana oyun döngüsü
def main():
    running = True
    selected_option = 0
    while running:
        draw_menu(selected_option)
        pygame.display.flip()

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_UP:  # Yukarı tuşu
                    selected_option = (selected_option - 1) % len(menu_options)
                elif event.key == pygame.K_DOWN:  # Aşağı tuşu
                    selected_option = (selected_option + 1) % len(menu_options)
                elif event.key == pygame.K_RETURN:  # Enter tuşu
                    if selected_option == 0:  # Yeni Oyun
                        play_game()
                    elif selected_option == 1:  # Çıkış
                        running = False
            elif event.type == pygame.MOUSEBUTTONDOWN:  # Fare tıklaması
                pos = pygame.mouse.get_pos()
                clicked_option = menu_selection(pos)
                if clicked_option != -1:
                    if clicked_option == 0:  # Yeni Oyun
                        play_game()
                    elif clicked_option == 1:  # Çıkış
                        running = False

    pygame.quit()

# Basit bir yapay zeka ile siyahın hamlesini yapma fonksiyonu
def ai_move(board):
    legal_moves = list(board.legal_moves)
    if legal_moves:
        move = random.choice(legal_moves)  # Rastgele geçerli bir hamle seç
        board.push(move)

# Oyun oynama fonksiyonu
def play_game():
    board = chess.Board()
    selected_square = None  # Seçilen kare
    running = True

    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.MOUSEBUTTONDOWN:  # Fare tıklaması
                mouse_x, mouse_y = event.pos
                col = mouse_x // SQUARE_SIZE
                row = 7 - (mouse_y // SQUARE_SIZE)  # Y ekseni ters
                clicked_square = chess.square(col, row)

                if selected_square is None:
                    # Eğer seçili kare yoksa, bu kareyi seç
                    piece = board.piece_at(clicked_square)
                    if piece is not None and piece.color == chess.WHITE:  # Sadece beyaz taşlarını seç
                        selected_square = clicked_square
                else:
                    # Seçili kareden hamle yap
                    if move_piece(board, selected_square, clicked_square):
                        selected_square = None  # Seçimi sıfırla
                        # Beyazın hamlesini yaptıktan sonra yalnızca siyahın hamlesini yap
                        if not board.is_game_over():
                            ai_move(board)
                    else:
                        # Geçersiz hamlede seçili kareyi sıfırla
                        selected_square = None

        screen.fill(WHITE)  # Ekranı temizle
        draw_board(board)  # Tahtayı çiz

        # Oyun bitiş kontrolü
        if board.is_checkmate():  # Şah mat durumu
            winner_color = not board.turn  # Kaybeden taraf
            show_winner(winner_color)  # Kazananı göster
            pygame.display.flip()  # Ekranı güncelle
            pygame.time.wait(3000)  # 3 saniye bekle
            running = False  # Oyunu bitir

        pygame.display.flip()  # Ekranı güncelle

if __name__ == "__main__":
    main()